In [1]:
import cv2
import os
from image_capture import ImageCapture
from face_recognition_model import FaceRecognitionModel
from customer import Customer

def capture_and_select_frames(num_images=10):
    """Captures and allows the user to select frames from the webcam."""
    image_capture = ImageCapture()
    images = image_capture.capture_images(num_images=num_images)
    
    saved_image_paths = []
    
    for idx, img in enumerate(images):
        filename = f"frame_{idx}.jpg"
        cv2.imwrite(filename, cv2.cvtColor(img, cv2.COLOR_RGB2BGR))
        saved_image_paths.append(filename)
        cv2.imshow(f"Captured Image {idx}", img)
    
    print("Press 'q' to finish capturing frames.")
    
    while True:
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break
    
    cv2.destroyAllWindows()
    
    return saved_image_paths

def register_customer(customer_id, locker_number, image_paths):
    """Registers a new customer with average face embedding."""
    model = FaceRecognitionModel()
    
    # Get face embeddings from images
    images = [cv2.imread(path) for path in image_paths]
    embeddings = model.get_embeddings(images)
    
    if len(embeddings) < 5:
        print("Please capture at least 5 frames.")
        return
    
    # Compute average embedding
    avg_embedding = model.average_embedding(embeddings)
    
    # Register customer with average embedding
    customer = Customer(customer_id, locker_number)
    customer.add_customer(avg_embedding)
    
    print(f"Customer {customer_id} registered successfully with locker {locker_number}")

if __name__ == "__main__":
    customer_id = input("Enter customer ID: ")
    locker_number = input("Enter locker number: ")
    
    print("Capturing frames...")
    image_paths = capture_and_select_frames(num_images=10)
    
    if len(image_paths) >= 5:
        print("Computing face embeddings and registering customer...")
        register_customer(customer_id, locker_number, image_paths)
    else:
        print("Not enough frames captured. Please try again.")


Capturing frames...
Press 'q' to finish capturing frames.


In [ ]:
import cv2
import dlib
import numpy as np
from scipy.spatial import distance as dist

class BlinkDetection:
    def __init__(self, shape_predictor_path):
        self.blink_threshold = 0.2
        self.consecutive_blinks = 3
        self.blink_counter = 0
        self.total_blinks = 0
        self.detector = dlib.get_frontal_face_detector()
        self.predictor = dlib.shape_predictor(shape_predictor_path)
        self.lStart, self.lEnd = 36, 42
        self.rStart, self.rEnd = 42, 48

    def eye_aspect_ratio(self, eye):
        A = dist.euclidean(eye[1], eye[5])
        B = dist.euclidean(eye[2], eye[4])
        C = dist.euclidean(eye[0], eye[3])
        ear = (A + B) / (2.0 * C)
        return ear

    def detect_blinks(self, shape):
        left_eye = shape[self.lStart:self.lEnd]
        right_eye = shape[self.rStart:self.rEnd]
        left_ear = self.eye_aspect_ratio(left_eye)
        right_ear = self.eye_aspect_ratio(right_eye)
        ear = (left_ear + right_ear) / 2.0
        return ear

    def start_detection(self):
        cap = cv2.VideoCapture(0)
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            faces = self.detector(gray)

            for face in faces:
                shape = self.predictor(gray, face)
                shape = np.array([[p.x, p.y] for p in shape.parts()])

                ear = self.detect_blinks(shape)

                if ear < self.blink_threshold:
                    self.blink_counter += 1
                else:
                    if self.blink_counter >= self.consecutive_blinks:
                        self.total_blinks += 1
                    self.blink_counter = 0

                cv2.putText(frame, "Blinks: {}".format(self.total_blinks), (10, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                cv2.putText(frame, "EAR: {:.2f}".format(ear), (300, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

            cv2.imshow("Blink Detection", frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()

if __name__ == "__main__":
    blink_detection = BlinkDetection("shape_predictor_68_face_landmarks.dat")
    blink_detection.start_detection()
